# South African COVID-19 Daily Hospital data API
### Data engineering project continued
#### Transformation phase 

--------------------------

The API and documentation is available on <a href='https://covidza-data.deta.dev/docs'>COVIDZA DATA API</a>

-----------------------------------

Looking for a data engineer, contact me:
 - [Linkedin]('https://linkedin.com/in/mpho-jan-kubeka')
 - [Github]('https://github.com/dataprojectswithMJ')
 - [Youtube]('https://www.youtube.com/channel/UClOP0fAisga04q3OB1iC4nQ')

-----------------------------

## Problem with Transformation

The biggest issues with this stage are:
   - Extracting the data from the PDFs in the correct formats
   - Setting up the correct data types

<img src='PDF%20Data.png' width='500'>

This screenshot shows the table on the PDF that contains the data that I will be using. Now I have to figure out formatting.

## Solution:
#### How did I get the data from the PDF documents?

I have 2 ways:
  1) __Programmatically__
      - This method means writing code that will read the PDF, annotate the table and then transform the data into the needed format.
      
      - There are 2 Python packages that I can use to annotate/read the table from the PDF:
        1) Camelot
           - Not the best.
        2) Tabula
           - Works fairly well but has issues remaining stable because of formatting styles in the PDFs.
           
  2) __Manually__
      - This method is much more accurate but very inefficient as each annotation is done by hand.
      - Remember __tabula__?
            There is a web browser tool that gives you a GUI to click and drag a shape over the data you need. From there onwards you can download the image in a CSV format for my use case(more formats available such as JSON, TSV etc).
            
<img src='tabula_annotation.png' width='500'> <img src='tabula_export.png' width='500'>

Because of the instability of the styles of formatting, I chose to do this _**manually**_. Though inefficient, it is extremely accurate.

I then export the annotated CSVs to a folder that I can iterate through.

----------------------------

My tech stack for this stage includes:
   - Python
        - Pandas
        - PyMONGO (MongoDB Python Wrapper)
        <br></br>
   - MongoDB

--------------------------

# Code

### Importing the needed packages

In [1]:
import glob
import pandas as pd

from datetime import datetime
from pymongo import MongoClient
import json
from bson.json_util import dumps,loads

I use __glob__ to find all the annotated CSVs and iterate over them. I use __Pandas__ instead of something like PySpark because there is little data to deal with rather than millions of data points.

### Configure MongoDB instance

In [3]:
uri = ' .... ' #uri is used to connect this wrapper to the database. Localhost for testing. Use ATLAS for cloud instance of MongoDB

client = MongoClient(uri)

db = client['COVIDAPI'] # Name of database where all the data will go into

links = db['records'] #Name of the collection (not table because this is a NoSQL database.

I use the same configurations as the extraction stage except for the collection name. This makes it so that the data can be found under 1 database but in different collections (tables in SQL terminology).

### Find the filenames

In [2]:
filename = glob.glob('tabula*') #Use a term that is common to all file names. Could even use '*.csv'. Which will extract all CSVs
filename[0] #Just to check if it was successful

'tabula_annotation.png'

### Now I can start reading the CSVs and doing some data transformations

In [ ]:
for files in filename:
    
    #Part 1:
    df = pd.read_csv(files)
    df1 = pd.DataFrame()

    print('Dataframe set up successful')
    
    #Part 2:
    intervals = [0, 3, 6, 9, 12, 15, 18, 21, 24]

    for x in intervals:
        df1 = df1.append(df.loc[x])
        
    df1.reset_index(drop='index', inplace=True)
    
    #Part 3:
    try:
        date = datetime.strptime(files.replace("tabula-", "").replace(".csv", "").replace("Tabula CSV's\\", ""),
                                 '%d %b %Y')
    except:
        date = datetime.strptime(files.replace("tabula-", "").replace(".csv", "").replace("Tabula CSV's\\", ""),
                                 '%d %B %Y')
    new_titles = []

    print(f'Working on {date} data')
    
    #Part 4:
    if df1.columns[2] == 'Admissions to Date':
        titles = ['Unnamed: 0', 'Facilities\rReporting', 'Admissions to Date',
                  'Died to\rDate', 'Discharged\rto date', 'Currently\rAdmitted',
                  'Currently\rin ICU', 'Currently\rVentilated', 'Currently\rOxygenated',
                  'Admissions in\rPrevious Day']
    else:
        titles = ['Unnamed: 0', 'Facilities\rReporting', 'Admissions\rto Date',
                  'Died to\rDate', 'Discharged\rto date', 'Currently\rAdmitted',
                  'Currently\rin ICU', 'Currently\rVentilated', 'Currently\rOxygenated',
                  'Admissions in\rPrevious Day']

    for x in titles:
        new_titles.append(x.replace("\r", "").replace(" ", ""))

    if df1.columns[0] == 'Unnamed: 0':
        new_titles[0] = 'Province'
    else:
        pass

    #Part 5:
    column_names = dict(zip(titles, new_titles))

    df1 = df1.rename(columns=column_names)
    print('Column name changes successful')


    for column in df1.columns:
        if column == 'AdmissionsinPreviousDay':
            df1.drop('AdmissionsinPreviousDay', axis=1, inplace=True)
        else:
            pass

    #Part 6:
    df1 = df1.append({'Province': 'National',
                      'FacilitiesReporting': df1['FacilitiesReporting'].sum(),
                      'AdmissionstoDate': df1['AdmissionstoDate'].sum(),
                      'DiedtoDate': df1['DiedtoDate'].sum(),
                      'Dischargedtodate': df1['Dischargedtodate'].sum(),
                      'CurrentlyAdmitted': df1['CurrentlyAdmitted'].sum(),
                      'CurrentlyinICU': df1['CurrentlyinICU'].sum(),
                      'CurrentlyVentilated': df1['CurrentlyVentilated'].sum(),
                      'CurrentlyOxygenated': df1['CurrentlyOxygenated'].sum()}, ignore_index=True)


    df1['Date'] = date
    
    print('Adding totals to dataframe successful..')
    
    #Part 7:    
    data = df1.to_dict(orient='records')
    print('Data conversion complete.')
    
    #Part 8:
    records.insert_many(data)

    #Part 9:
    move_file = files.replace("Tabula CSVs\\", "")
    
    initial_file_path = ' ..... '
    move_file_path

    os.replace(f"{initial_file_path}{move_file}",
               f"{move_file_path}{move_file}")

    print('File moved Successfully!')

    print(f'Upload to DB for {date} Successful')
    print('----------------------------------------------------')


There is a lot going on in the above cell (98 lines of code in total). I seperated the code into parts that reader can use to understand the code.

## PART: 
  1) I create 2 dataframes (1 for entering the data and another for storing the cleaned data.
  <br></br>
  2) The most important values are the Provinces. This stage uses their index from the old dataframe to the new one.
    <br></br>
  3) This stage I use the try and except method to configure the acceptable datetime format. Remeber the __filename__ varible from glob, it stores the names of the files and in the extraction stage I stored the date as the file name. Now that action comes into play. I also replaced extra text (ie tabula, .csv etc) to only keep the __date__ from the name.
    <br></br>
  4) I stored the column names in a list called __titles__ and upon analysing the it, I noticed that sometimes the column 'Admissions to Date' is formatted differently. Fortunately I discovered only 2 ways this is done either with whitespaces or sometimes with __"\r"__ in between the text, so it is fairly easy to deal with. That list is iterated through and then the column names are cleaned free of whitespaces and characters stored in a new list called __new_titles__. Sometimes the first column is stored as __'Unnamed: 0'__ so I switch it to __'Province'__.
    <br></br>
 5) Now I zip together the 2 lists (titles and new_titles) together in order to rename the dataframe columns to clean column names. There is a __'AdmissionsinPreviousDay'__ column which is not necessary for this project so I remove it.
    <br></br>
 6) The only thing missing now is the __Total__ count for the record. Using a dataframe append function, I am able to sum up each column and add the sum to the last row which named __'National'__ because the sum is all provinces combined. I also add the date to each row. This is so that when each record is loaded to the database it will have a date included and can be filtered by it.
     <br></br>
 7) Now I convert the dataframe to a dictionary using the 'to_dict' function with records orientation
     <br></br>
 ### Part 8 is the *LOADING* step in this ETL process
 8) I use __'insert_many'__ because the __'to_dict'__ function stores the each row as a dictionary. So the dictionary is a list of dictionaries. The reason why I need them in a dictionary is because PyMongo uses dictionaries to insert new data.
     <br></br>
 9) This is where I conclude the script. I have 2 variables __initial_file_path__ and __move_file_path__. This is for moving the files that the code is done with to another location. 